## Parte 1 ● Ejecutar los scripts con una muestra relativamente pequeña de cada uno de losendpoints y verificar que todo funcione correctamente.

https://www.ree.es/es/datos/apidatos




NUEVO CODIGO DE EXTRACCION

In [ ]:
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from datetime import datetime

In [ ]:
# Versiones

print(f"numpy=={np.__version__}")
print(f"pandas=={pd.__version__}")
print(f"requests=={requests.__version__}")

In [8]:
# Constantes comunes
URL = "https://apidatos.ree.es" # URL base de la API
HEADERS = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Host": "apidatos.ree.es",
    "User-Agent": "Mozilla/5.0"
}

# Diccionario de geo_ids y nombres de regiones
region_ids = {
    "8741": "peninsular",
    "8742": "canarias",
    "8743": "baleares",
    "8744": "ceuta",
    "8745": "melilla",
    "4": "andalucia",
    "5": "aragon",
    "6": "cantabria",
    "7": "castilla la mancha",
    "8": "castilla y leon",
    "9": "cataluña",
    "10": "pais vasco",
    "11": "principado de asturias",
    "13": "comunidad de madrid",
    "14": "comunidad de navarra",
    "15": "comunidad de valenciana",
    "16": "extremadura",
    "17": "galicia",
    "20": "la rioja",
    "21": "region de murcia"
}

In [ ]:
filtro_balance = '/es/datos/balance/balance-electrico'

filtro_demanda = '/es/datos/demanda/evolucion'
filtro_general = '/es/datos/demanda/ire-general'
filtro_industria = '/es/datos/demanda/ire-industria'
filtro_servicios = '/es/datos/demanda/ire-servicios'

filtro_generacion = "/es/datos/generacion/estructura-generacion"
filtro_renovable = "/es/datos/generacion/evolucion-renovable-no-renovable"

In [ ]:
def Balance_electrico(filtro_balance, URL, HEADERS):

    filtro_balance

    endpoint = f"{URL}/{filtro_balance}"

    # Inicializamos un DataFrame vacío para almacenar los datos
    df_balance = pd.DataFrame()

    # Iteramos sobe los años de 2019 a 2024 y extraemos los datos de balance
    # de cada región para cada año
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "day"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "day"

        print(f"Extrayendo datos de balance del año {year}...")

        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            data = response.json()
            included = data.get('included', [])

            registros = []
            # Extraemos los datos de balance
            # para cada grupo de energía
            # y cada indicador
            for grupo in included:
                group_name = grupo['attributes']['title']
                contenidos = grupo['attributes'].get('content', [])

                for item in contenidos:
                    indicador = item['attributes']['title']
                    valores = item['attributes'].get('values', [])
                    # Extraemos los valores para cada indicador
                    # y los almacenamos en el DataFrame
                    for punto in valores:
                        registros.append({
                            'fecha': punto['datetime'],
                            'valor': punto['value'],
                            'tipo': indicador,
                            'energia': group_name,
                            'region': region_name,
                        })
            # Convertimos los registros a un DataFrame
            # y lo concatenamos al DataFrame principal
            df_balance_year = pd.DataFrame(registros)
            df_balance = pd.concat([df_balance, df_balance_year], ignore_index=True)

    print('Datod de balance entre 2015 y 2025 extraidos:')
    display(df_balance.head())

    return df_balance


In [ ]:
def demanda_evolucion(filtro_demanda, URL, HEADERS):

    filtro_demanda

    endpoint = f"{URL}/{filtro_demanda}"
    # Inicializamos un df vacío para almacenar los datos
    df_demanda = pd.DataFrame()
    # Iteramos sobre los años y regiones
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "day"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "day"

        print(f"Extrayendo datos de demanda del año {year}...")

        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            data = response.json()
            included = data.get('included', [])

            registros = []

            for serie in included:
                indicador = serie['attributes']['title']
                valores = serie['attributes'].get('values', [])
                # Extraer los valores de la serie
                for punto in valores:
                    registros.append({
                        'fecha': punto['datetime'],
                        'valor': punto['value'],
                        'indicador': indicador,
                        'region': region_name,
                        'año': year
                    })

            df_demanda_year = pd.DataFrame(registros)
            df_demanda = pd.concat([df_demanda, df_demanda_year], ignore_index=True)

    print("Datos de demanda extraídos entre el 2015 y el 2025:")
    display(df_demanda)

    return df_demanda

In [ ]:
def demanda_ire_general(filtro_general, URL, HEADERS):
    
    filtro_general

    endpoint = f"{URL}/{filtro_demanda}"

    # Inicializamos un DataFrame vacío
    df_ire_general = pd.DataFrame()

    # Iteramos sobre los años y regiones
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "month"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "month"

        print(f"Extrayendo datos IRE General del año {year}...")

        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            if response.status_code != 200:
                print(f"Error {response.status_code} para región: {region_name} ({geo_id}), año {year}")
                continue

            data = response.json()
            included = data.get('included', [])

            registros = []

            for serie in included:
                indicador = serie['attributes']['title']
                valores = serie['attributes'].get('values', [])
                for punto in valores:
                    registros.append({
                        'fecha': punto['datetime'],
                        'valor': punto['value'],
                        'porcentaje': punto.get('percentage'),
                        'indicador': indicador,
                        'region': region_name,
                        
                    })

            df_ire_year = pd.DataFrame(registros)
            df_ire_general = pd.concat([df_ire_general, df_ire_year], ignore_index=True)

    print("Datos IRE General extraídos entre el 2015 y el 2025:")
    display(df_ire_general)

    return df_ire_general

In [ ]:
def demanda_ire_industria(filtro_industria, URL, HEADERS):

    filtro_industria

    endpoint = f"{URL}/{filtro_demanda}"

    # Inicializamos un DataFrame vacío
    df_ire_industria = pd.DataFrame()

    # Iteramos sobre los años y regiones
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "month"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "month"

        print(f"Extrayendo datos IRE industria del año {year}...")

        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            if response.status_code != 200:
                print(f"Error {response.status_code} para región: {region_name} ({geo_id}), año {year}")
                continue

            data = response.json()
            included = data.get('included', [])

            registros = []

            for serie in included:
                indicador = serie['attributes']['title']
                valores = serie['attributes'].get('values', [])
                for punto in valores:
                    registros.append({
                        'fecha': punto['datetime'],
                        'valor': punto['value'],
                        'porcentaje': punto.get('percentage'),
                        'indicador': indicador,
                        'region': region_name,
                        'año': year
                    })

            df_ire_industria_year = pd.DataFrame(registros)
            df_ire_industria = pd.concat([df_ire_industria, df_ire_industria_year], ignore_index=True)

    print("Datos IRE industria anual extraídos entre el 2015 y el 2025:")
    display(df_ire_industria)

    return df_ire_industria

In [30]:
def demanda_ire_servicios(filtro_servicios, URL, HEADERS):

    filtro_servicios

    endpoint = f"{URL}/{filtro_demanda}"

    # Inicializamos un DataFrame vacío
    df_ire_servicios = pd.DataFrame()

    # Iteramos sobre los años y regiones
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "month"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "month"

        print(f"Extrayendo datos IRE servicios del año {year}...")

        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            if response.status_code != 200:
                print(f"Error {response.status_code} para región: {region_name} ({geo_id}), año {year}")
                continue

            data = response.json()
            included = data.get('included', [])

            registros = []

            for serie in included:
                indicador = serie['attributes']['title']
                valores = serie['attributes'].get('values', [])
                for punto in valores:
                    registros.append({
                        'fecha': punto['datetime'],
                        'valor': punto['value'],
                        'porcentaje': punto.get('percentage'),
                        'indicador': indicador,
                        'region': region_name,
                        'año': year
                    })

            df_ire_servicios_year = pd.DataFrame(registros)
            df_ire_servicios = pd.concat([df_ire_servicios, df_ire_servicios_year], ignore_index=True)

    print("Datos IRE servicios extraídos entre el 2015 y el 2025:")
    display(df_ire_servicios)

In [32]:
def generacion(filtro_generacion, URL, HEADERS):

    filtro_generacion

    endpoint = f"{URL}{filtro_generacion}"

    df_generacion = pd.DataFrame()
    ## Iterar sobre los años
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "day"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "day"

        print(f"Extrayendo datos de estructura-generacion del año {year}...")
        # Iterar sobre los geo_ids y nombres de regiones
        # para cada año y geo_id
        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            if response.status_code == 200:
                try:
                    data = response.json()
                    included = data.get('included', [])

                    if included:
                        registros = []

                        # Iterar sobre los grupos de datos en 'included'
                        for grupo in included:
                            group_name = grupo['attributes'].get('title', 'Desconocido')
                            group_type = grupo['attributes'].get('type', 'Desconocido')
                            contenidos = grupo['attributes'].get('values', [])

                            # Iterar sobre los valores de generación
                            for punto in contenidos:
                                registros.append({
                                    'fecha': punto.get('datetime', 'Desconocido'),
                                    'valor': punto.get('value', 0),
                                    'porcentaje': punto.get('percentage', 0),
                                    'indicador': group_name,
                                    'region': region_name,
                                    'tipo': group_type,
                                    'año': year
                                })

                        df_generacion_year = pd.DataFrame(registros)
                        df_generacion = pd.concat([df_generacion, df_generacion_year], ignore_index=True)

                except Exception as e:
                    print("Contenido de la respuesta:", response.text)

            else:
                print(f"Error en la solicitud para {region_name} {year}. Código: {response.status_code}")

    print('Datos de estructura-generacion del 2015 al 2025:')
    display(df_generacion)

    return df_generacion

In [ ]:
def renov_norenov(filtro_renovable, URL, HEADERS):

    filtro_renovable

    endpoint = f"{URL}{filtro_generacion}"

    df_renovable = pd.DataFrame()
    ## Iterar sobre los años
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "day"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "day"

        print(f"Extrayendo datos de evolucion renovable/No renovable del año {year}...")
        # Iterar sobre los geo_ids y nombres de regiones
        # para cada año y geo_id
        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            if response.status_code == 200:
                try:
                    data = response.json()
                    included = data.get('included', [])

                    if included:
                        registros = []

                        # Iterar sobre los grupos de datos en 'included'
                        for grupo in included:
                            group_name = grupo['attributes'].get('title', 'Desconocido')
                            group_type = grupo['attributes'].get('type', 'Desconocido')
                            contenidos = grupo['attributes'].get('values', [])

                            # Iterar sobre los valores de generación
                            for punto in contenidos:
                                registros.append({
                                    'fecha': punto.get('datetime', 'Desconocido'),
                                    'valor': punto.get('value', 0),
                                    'porcentaje': punto.get('percentage', 0),
                                    'indicador': group_name,
                                    'region': region_name,
                                    'tipo': group_type,
                                    'año': year
                                })

                        df_renovable_year = pd.DataFrame(registros)
                        df_renovable = pd.concat([df_renovable, df_renovable_year], ignore_index=True)

                except Exception as e:
                    print("Contenido de la respuesta:", response.text)

            else:
                print(f"Error en la solicitud para {region_name} {year}. Código: {response.status_code}")

    print('Datos de generación del 2015 al 2025:')
    display(df_renovable)

    return df_renovable

In [36]:
def fronteras(URL, HEADERS):

    # Lista de países
    lista_paises = ['francia-frontera', 'portugal-frontera', 'marruecos-frontera', 'andorra-frontera']

    datos_intercambios = []

    for pais in lista_paises:
        print(f'Consultando datos de {pais}...')
        # Endpoint para intercambios
        filtro_intercambio = f"/es/datos/intercambios/{pais}"
        endpoint = f"{URL}{filtro_intercambio}"
        # Iterar sobre los años y extraer datos de cada año
        # y cada país para el año 2015 al 2025
        for year in range(2015, 2026):

            if year < 2025:
                start_date = f"{year}-01-01T00:00"
                end_date = f"{year}-12-31T23:59"
                TIME_TRUNC = "day"
            else:
                start_date = f"{year}-01-01T00:00"
                end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
                TIME_TRUNC = "day"

            print(f"Extrayendo datos del año {year}...")

            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            if response.status_code == 200:
                data = response.json()
                # Verificar si hay datos disponibles
                valores = data["included"][0]["attributes"].get("values", [])
                if valores:
                    df = pd.DataFrame(valores)
                    df["pais"] = pais
                    df["año"] = year
                    datos_intercambios.append(df)
            else:
                print(f"Error en la solicitud para {pais} en {year}. Código: {response.status_code}")

    # Unir todos los datos en un único DataFrame
    df_fronteras = pd.concat(datos_intercambios, ignore_index=True)

    print(f"Datos de intercambios entre paises del 2015 al 2025:")
    display(df_fronteras)

    return df_fronteras

In [ ]:
def extraccion():
    """
    Función para extraer datos de la API de REE.
    """
    print('Extrayendo datos del Balance Eléctrico...')
    df_balance = Balance_electrico(filtro_balance, URL, HEADERS)
    
    print('Extrayendo datos de la Demanda Eléctrica...')
    df_demanda = demanda_evolucion(filtro_demanda, URL, HEADERS)
    print('Extrayendo datos de la IRE General...')
    df_ire_general = demanda_ire_general(filtro_general, URL, HEADERS)
    print('Extrayendo datos de la IRE Industria...')
    df_ire_industria = demanda_ire_industria(filtro_industria, URL, HEADERS)
    print('Extrayendo datos de la IRE Servicios...')
    df_ire_servicios = demanda_ire_servicios(filtro_servicios, URL, HEADERS)

    print('Extrayendo datos de la Generación Eléctrica...')
    df_generacion = generacion(filtro_generacion, URL, HEADERS)
    print('Extrayendo datos de la Generación Renovable y No Renovable...')
    df_renovable = renov_norenov(filtro_renovable, URL, HEADERS)

    print('Extrayendo datos de los intercambios entre países...')
    df_fronteras = fronteras(URL, HEADERS)

    return {
        "balance": df_balance,
        "demanda": df_demanda,
        "ire_general": df_ire_general,
        "ire_industria": df_ire_industria,
        "ire_servicios": df_ire_servicios,
        "generacion": df_generacion,
        "renovable": df_renovable,
        "fronteras": df_fronteras,
    }

In [39]:
dataframes = extraccion()

Extrayendo datos de balance del año 2015...
Extrayendo datos de balance del año 2016...
Extrayendo datos de balance del año 2017...
Extrayendo datos de balance del año 2018...
Extrayendo datos de balance del año 2019...
Extrayendo datos de balance del año 2020...
Extrayendo datos de balance del año 2021...
Extrayendo datos de balance del año 2022...
Extrayendo datos de balance del año 2023...
Extrayendo datos de balance del año 2024...
Extrayendo datos de balance del año 2025...
Datod de balance entre 2019 y 2024 extraidos:


,fecha,valor,tipo,energia,region
0,2015-01-01T00:00:00.000+01:00,71096.101,Hidráulica,Renovable,peninsular
1,2015-01-02T00:00:00.000+01:00,101890.114,Hidráulica,Renovable,peninsular
2,2015-01-03T00:00:00.000+01:00,82919.726,Hidráulica,Renovable,peninsular
3,2015-01-04T00:00:00.000+01:00,69056.368,Hidráulica,Renovable,peninsular
4,2015-01-05T00:00:00.000+01:00,104975.898,Hidráulica,Renovable,peninsular


Extrayendo datos de demanda del año 2015...
Extrayendo datos de demanda del año 2016...
Extrayendo datos de demanda del año 2017...
Extrayendo datos de demanda del año 2018...
Extrayendo datos de demanda del año 2019...
Extrayendo datos de demanda del año 2020...
Extrayendo datos de demanda del año 2021...
Extrayendo datos de demanda del año 2022...
Extrayendo datos de demanda del año 2023...
Extrayendo datos de demanda del año 2024...
Extrayendo datos de demanda del año 2025...
Datos de demanda extraídos entre el 2019 y el 2024:


,fecha,valor,indicador,region,año
0,2015-01-01T00:00:00.000+01:00,603961.474,Demanda,peninsular,2015
1,2015-01-02T00:00:00.000+01:00,715038.679,Demanda,peninsular,2015
2,2015-01-03T00:00:00.000+01:00,698648.512,Demanda,peninsular,2015
3,2015-01-04T00:00:00.000+01:00,663533.254,Demanda,peninsular,2015
4,2015-01-05T00:00:00.000+01:00,714784.323,Demanda,peninsular,2015
...,...,...,...,...,...
75315,2025-04-19T00:00:00.000+02:00,548294.130,Demanda,region de murcia,2025
75316,2025-04-20T00:00:00.000+02:00,537763.459,Demanda,region de murcia,2025
75317,2025-04-21T00:00:00.000+02:00,586636.561,Demanda,region de murcia,2025
75318,2025-04-22T00:00:00.000+02:00,658548.194,Demanda,region de murcia,2025


Extrayendo datos IRE General del año 2015...
Extrayendo datos IRE General del año 2016...
Extrayendo datos IRE General del año 2017...
Extrayendo datos IRE General del año 2018...
Extrayendo datos IRE General del año 2019...
Extrayendo datos IRE General del año 2020...
Extrayendo datos IRE General del año 2021...
Extrayendo datos IRE General del año 2022...
Extrayendo datos IRE General del año 2023...
Extrayendo datos IRE General del año 2024...
Extrayendo datos IRE General del año 2025...
Datos IRE General extraídos entre el 2015 y el 2025:


...

,fecha,valor,porcentaje,indicador,region,tipo,año
0,2015-01-01T00:00:00.000+01:00,71096.101,0.114430,Hidráulica,peninsular,Renovable,2015
1,2015-01-02T00:00:00.000+01:00,101890.114,0.147343,Hidráulica,peninsular,Renovable,2015
2,2015-01-03T00:00:00.000+01:00,82919.726,0.117828,Hidráulica,peninsular,Renovable,2015
3,2015-01-04T00:00:00.000+01:00,69056.368,0.101494,Hidráulica,peninsular,Renovable,2015
4,2015-01-05T00:00:00.000+01:00,104975.898,0.145461,Hidráulica,peninsular,Renovable,2015
...,...,...,...,...,...,...,...
1205635,2025-04-19T00:00:00.000+02:00,646861.730,1.000000,Generación total,region de murcia,Generación total,2025
1205636,2025-04-20T00:00:00.000+02:00,585391.759,1.000000,Generación total,region de murcia,Generación total,2025
1205637,2025-04-21T00:00:00.000+02:00,625630.361,1.000000,Generación total,region de murcia,Generación total,2025
1205638,2025-04-22T00:00:00.000+02:00,703804.434,1.000000,Generación total,region de murcia,Generación total,2025


Extrayendo datos de evolucion renovable/No renovable del año 2015...
Extrayendo datos de evolucion renovable/No renovable del año 2016...
Extrayendo datos de evolucion renovable/No renovable del año 2017...
Extrayendo datos de evolucion renovable/No renovable del año 2018...
Extrayendo datos de evolucion renovable/No renovable del año 2019...
Extrayendo datos de evolucion renovable/No renovable del año 2020...
Extrayendo datos de evolucion renovable/No renovable del año 2021...
Extrayendo datos de evolucion renovable/No renovable del año 2022...
Extrayendo datos de evolucion renovable/No renovable del año 2023...
Extrayendo datos de evolucion renovable/No renovable del año 2024...
Extrayendo datos de evolucion renovable/No renovable del año 2025...
Datos de generación del 2015 al 2025:


,fecha,valor,porcentaje,indicador,region,tipo,año
0,2015-01-01T00:00:00.000+01:00,71096.101,0.114430,Hidráulica,peninsular,Renovable,2015
1,2015-01-02T00:00:00.000+01:00,101890.114,0.147343,Hidráulica,peninsular,Renovable,2015
2,2015-01-03T00:00:00.000+01:00,82919.726,0.117828,Hidráulica,peninsular,Renovable,2015
3,2015-01-04T00:00:00.000+01:00,69056.368,0.101494,Hidráulica,peninsular,Renovable,2015
4,2015-01-05T00:00:00.000+01:00,104975.898,0.145461,Hidráulica,peninsular,Renovable,2015


Consultando datos de francia-frontera...
Extrayendo datos del año 2015...
Extrayendo datos del año 2016...
Extrayendo datos del año 2017...
Extrayendo datos del año 2018...
Extrayendo datos del año 2019...
Extrayendo datos del año 2020...
Extrayendo datos del año 2021...
Extrayendo datos del año 2022...
Extrayendo datos del año 2023...
Extrayendo datos del año 2024...
Extrayendo datos del año 2025...
Consultando datos de portugal-frontera...
Extrayendo datos del año 2015...
Extrayendo datos del año 2016...
Extrayendo datos del año 2017...
Extrayendo datos del año 2018...
Extrayendo datos del año 2019...
Extrayendo datos del año 2020...
Extrayendo datos del año 2021...
Extrayendo datos del año 2022...
Extrayendo datos del año 2023...
Extrayendo datos del año 2024...
Extrayendo datos del año 2025...
Consultando datos de marruecos-frontera...
Extrayendo datos del año 2015...
Extrayendo datos del año 2016...
Extrayendo datos del año 2017...
Extrayendo datos del año 2018...
Extrayendo datos

,value,percentage,datetime,pais,año
0,-3384.261,0.122156,2015-01-01T00:00:00.000+01:00,francia-frontera,2015
1,-60.199,0.002045,2015-01-02T00:00:00.000+01:00,francia-frontera,2015
2,-19.245,0.000649,2015-01-03T00:00:00.000+01:00,francia-frontera,2015
3,-31.131,0.000995,2015-01-04T00:00:00.000+01:00,francia-frontera,2015
4,-2.958,0.000098,2015-01-05T00:00:00.000+01:00,francia-frontera,2015
...,...,...,...,...,...
14967,-425.510,1.000000,2025-03-27T00:00:00.000+01:00,andorra-frontera,2025
14968,-424.820,1.000000,2025-03-28T00:00:00.000+01:00,andorra-frontera,2025
14969,-423.500,1.000000,2025-03-29T00:00:00.000+01:00,andorra-frontera,2025
14970,-361.800,1.000000,2025-03-30T00:00:00.000+01:00,andorra-frontera,2025
